# Clustering Chunks
Goals:
- Equal representation of patterns in the ML models
- To figure out more about noisy chunks and ways to throw away. However, there is no "clear" description of what can be attributed as noise. 

In [31]:
%matplotlib inline

from sklearn import cluster
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

import scipy.cluster.hierarchy as hac
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import calendar
import common

In [65]:
def distance_func(t1, t2, *args):
    return fastdtw(t1, t2, dist=euclidean)[0]

# replace nulls with rolling mean or near ones
def replace_nulls(df, cols=[]):
    cols = cols or df.columns
    mean = df[cols].rolling(5, min_periods=1).mean()
    df[cols] = df[cols].fillna(mean)
    df[cols] = df[cols].fillna(method="pad")
    return df

In [66]:
df = common.load_df("../data/insead", "*.csv")
sample = df["2016-01":"2016-03"]
sample = replace_nulls(sample)

C:\Anaconda2\envs\py3\lib\site-packages\pandas\core\frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [67]:
# this can be done neatly using pandas..
def create_chunks(df, field="cwshdr"):
    periods = []    
    for y in df.index.year.unique():
        for m in df.index.month.unique():
            _, last = calendar.monthrange(y, m)
            first = 1
            for d in range(first, last+1):
                today = "{}-{}-{}".format(y, m, d)
                if df[today:today].shape[0] > 0:
                    periods.append(today)
    chunks = []
    for p in periods:
        field_data = df[field][p:p].values
        if field_data.shape[0] >= 20 * 60:
            chunks.append(field_data[:20*60])
    
    return np.array(chunks)

chunks = create_chunks(df)

In [69]:
cluster = hac.fclusterdata(chunks, 1.0, metric=distance_func)

ValueError: array must not contain infs or NaNs